In [69]:
from collections import Counter
import random

import numpy as np
import pyarrow.dataset as ds
import duckdb
import pandas as pd
import numpy as np
import seaborn as sns
import plotly as pl
import plotly.express as px
import json

from plotly.graph_objs import Figure

DIMENSIONS = {
    "Região geográfica": "NO_REGIAO",
    "Sigla da Unidade da Federeção": "SG_UF",
    "Unidade da Federação": "NO_UF",
    "Mesorregião": "NO_MESORREGIAO",
    "Microrregião": "NO_MICRORREGIAO",
    "Município": "NO_MUNICIPIO"
}

con = duckdb.connect('../file.db')

microdados = ds.dataset("../data/transformed/microdados.parquet", format="parquet", partitioning="hive")
con.register("microdados", microdados)

# indicadores = {}
# for indicador in INDICADORES:
#     indicadores[indicador] = ds.dataset(f"../data/transformed/indicadores/{indicador}.parquet", format="parquet", partitioning="hive")
#     con.register(indicador, indicadores[indicador])

In [70]:
dimension = "Município"
df = con.execute(f"""
                select
                    NU_ANO_CENSO as 'Ano',
                    {DIMENSIONS[dimension]} as '{dimension}',
                    round(cast(count(*) filter (where IN_AGUA_REDE_PUBLICA) as float) / count(*) * 100, 3) as 'Nível de acesso em %',
                    --round(cast(count(*) filter (where IN_AGUA_POCO_ARTESIANO) as float) / count(*), 3) as 'Poço artesiano',
                    --round(cast(count(*) filter (where IN_AGUA_CACIMBA) as float) / count(*), 3) as 'Cacimba/Cisterna/Poço',
                    --round(cast(count(*) filter (where IN_AGUA_FONTE_RIO) as float) / count(*), 3) as 'Fonte/Rio/Igarapé/Riacho/Córrego',
                    --round(cast(count(*) filter (where IN_AGUA_INEXISTENTE) as float) / count(*), 3) as 'Não há abastecimento de água',
--                     round(
--                         cast(count(*) filter
--                             (where not (IN_AGUA_REDE_PUBLICA or IN_AGUA_POCO_ARTESIANO or IN_AGUA_CACIMBA or IN_AGUA_FONTE_RIO or IN_AGUA_INEXISTENTE ))
--                             as float)
--                         / count(*)
--                     , 3) as 'Sem informações'
                from microdados
                group by NU_ANO_CENSO, {DIMENSIONS[dimension]}
                order by 1, 2
            """
).df()

In [71]:
df.head()

,Ano,Município,Nível de acesso em %
0,2016,Abadia de Goiás,88.889000
1,2016,Abadia dos Dourados,55.556000
2,2016,Abadiânia,47.826000
3,2016,Abaetetuba,4.587000
4,2016,Abaeté,72.726997


In [67]:
with open("../data/geo/municipio.json", encoding="latin1") as f:
    uf_json = json.load(f)

In [68]:
uf_json["features"][0]["properties"]

{'GEOCODIGO': 1200336.0,
 'NOME': 'Mâncio Lima',
 'UF': 'AC',
 'ID_UF': 12.0,
 'REGIAO': 'Norte',
 'MESOREGIAO': 'VALE DO JURUA',
 'MICROREGIA': 'CRUZEIRO DO SUL',
 'LATITUDE': '-7.614',
 'LONGITUDE': '-72.896'}

In [72]:
fig = px.choropleth_mapbox(
    df,
    geojson=uf_json,
    color="Nível de acesso em %",
    locations=dimension,
    range_color=(0, 100),
    mapbox_style = "white-bg",
    featureidkey="properties.NOME",
    center = {"lat": -14, "lon": -55},
    animation_frame="Ano",
    zoom=3,
    width=750,
    height=750
)

In [37]:
fig.show()

In [73]:
fig.write_html("test.html")

In [40]:
df.iloc[0]

Ano                                 2016
Mesorregião             Agreste Alagoano
Nível de acesso em %              50.112
Name: 0, dtype: object